In [1]:
import sys, os

env_root = '/N/project/baby_vision_curriculum/pythonenvs/hfenv/lib/python3.10/site-packages/'
sys.path.insert(0, env_root)


In [2]:
import numpy as np
import torch, torchvision
from torchvision import transforms as tr
from tqdm import tqdm
from pathlib import Path
# import math
import argparse

import torch.distributed as dist
from torch.utils.data.distributed import DistributedSampler
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.multiprocessing as mp

# from copy import deepcopy

import cv2
from itertools import chain

In [3]:

sys.path.insert(0, '../predictive/scripts')

In [4]:
import vision_transformer as vit
from helper import load_checkpoint

import logging
from tensors import trunc_normal_

In [8]:
def init_weights(m):
    if isinstance(m, torch.nn.Linear):
        trunc_normal_(m.weight, std=0.02)
        if m.bias is not None:
            torch.nn.init.constant_(m.bias, 0)
    elif isinstance(m, torch.nn.LayerNorm):
        torch.nn.init.constant_(m.bias, 0)
        torch.nn.init.constant_(m.weight, 1.0)
            
def get_untrained_model(
    patch_size=16,
    tubelet_size=1,
    num_frames=1,
    model_name='vit_base',
    image_size=224,
    pred_depth=6,
    pred_emb_dim=384
):
#     encoder = vit_base(
    encoder = vit.__dict__[model_name](
        img_size=[image_size],
        patch_size=patch_size,
        num_frames=num_frames,
        tubelet_size=tubelet_size)
    
    for m in encoder.modules():
        init_weights(m)

#     encoder.to(device)
    # logger.info(encoder)
    return encoder

def get_model(args):
    
    encoder = get_untrained_model(num_frames=args.num_frames, tubelet_size=args.tubelet_size)
    
    load_path = args.init_checkpoint_path
    if load_path!='na':
        print('load_path:',load_path)
        encoder, _, _, _, scaler, _ = load_checkpoint(r_path=load_path,
            encoder=encoder, predictor=None, target_encoder=None, opt=None, scaler=None)
    
    return encoder

In [9]:
class Args:
    def __init__(self, **kwargs):
        for key, value in kwargs.items():
            setattr(self, key, value)

In [78]:
init_checkpoint_path='/N/project/baby_vision_curriculum/trained_models/predictive/v0/jul21/model_dev_1_g0_default_0_261.pth.tar'
# '/N/project/baby_vision_curriculum/trained_models/predictive/v0/jul21/model_dev_1_g0_default_0_261.pth.tar'
# '/N/project/baby_vision_curriculum/trained_models/predictive/v0/jul24/jepa-latest.pth.tar'
# 'na'

args = Args(tubelet_size=1,
            num_frames=6,
           init_checkpoint_path=init_checkpoint_path)
model_source = get_model(args)

load_path: /N/project/baby_vision_curriculum/trained_models/predictive/v0/jul21/model_dev_1_g0_default_0_261.pth.tar
INFO:root:loaded pretrained encoder from epoch 1 with msg: <All keys matched successfully>
INFO:root:read-path: /N/project/baby_vision_curriculum/trained_models/predictive/v0/jul21/model_dev_1_g0_default_0_261.pth.tar


In [73]:
encoder = get_untrained_model(num_frames=args.num_frames, tubelet_size=args.tubelet_size)

In [74]:
checkpoint = torch.load(init_checkpoint_path)#, map_location=torch.device('cpu'))

In [75]:
epoch = checkpoint['epoch']
pretrained_dict = checkpoint['encoder']
msg = encoder.load_state_dict(pretrained_dict)
# checkpoint['encoder']

In [81]:
xtt = torch.rand(2, 6, 3, 224,224)

In [82]:
ytt = model_source(xtt, masks=None)

In [83]:
ytt.shape

torch.Size([2, 1176, 768])

In [30]:
784*768

602112

In [84]:
ytt.mean(1).shape

torch.Size([2, 768])